In [ ]:
import os
import pandas as pd
import numpy as np

def process_csv_files(folder_path):
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)

            # Convert 'Date' column to datetime format
            df['Date'] = pd.to_datetime(df['Date'])

            # Calculate daily returns as percentage
            df['daily_return'] = df['Adj Close'].pct_change() * 100

            # Compute monthly mean of daily returns
            monthly_return_mean = df.groupby(df['Date'].dt.to_period('M'))['daily_return'].mean()
            df['monthly_mean'] = df['Date'].dt.to_period('M').map(monthly_return_mean)

            # Compute monthly RMSE of daily returns vs monthly mean
            rmse_map = df.groupby(df['Date'].dt.to_period('M')).apply(
                lambda x: np.sqrt(np.mean((x['daily_return'] - x['monthly_mean']) ** 2))
            )
            df['RMSE'] = df['Date'].dt.to_period('M').map(rmse_map)

            # Compute monthly mean of volume
            monthly_volume_mean = df.groupby(df['Date'].dt.to_period('M'))['Volume'].mean()
            df['Vol_month_mean'] = df['Date'].dt.to_period('M').map(monthly_volume_mean)

            # Save the updated DataFrame back to the same CSV file
            df.to_csv(file_path, index=False)

# Apply processing to both 'stocks' and 'etfs' folders
for folder in ['Data/stocks', 'Data/etfs']:
    process_csv_files(folder)
